# COMP5318 Assignment 1: Classification

### Group number: 64  , SID1: 520087343 , SID2: 520087479  

In [1]:
# Import all libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Load Breast Cancer Wisconsin Dataset
data = pd.read_csv("breast-cancer-wisconsin.csv")

In [3]:
data.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,class
0,5,1,1,1,2,1,3,1,1,class1
1,5,4,4,5,7,10,3,2,1,class1
2,3,1,1,1,2,2,3,1,1,class1
3,6,8,8,1,3,4,3,7,1,class1
4,4,1,1,3,2,1,3,1,1,class1


In [4]:
#See which column exists ? needs to be replaced
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Clump Thickness               699 non-null    int64 
 1   Uniformity of Cell Size       699 non-null    int64 
 2   Uniformity of Cell Shape      699 non-null    int64 
 3   Marginal Adhesion             699 non-null    int64 
 4    Single Epithelial Cell Size  699 non-null    int64 
 5   Bare Nuclei                   699 non-null    object
 6   Bland Chromatin               699 non-null    int64 
 7   Normal Nucleoli               699 non-null    int64 
 8   Mitoses                       699 non-null    int64 
 9   class                         699 non-null    object
dtypes: int64(8), object(2)
memory usage: 54.7+ KB


In [5]:
#Define a function if it is a number, it will be converted to int type, if it is '?', it will be converted to np.nan
def to_number(x):
    try:
        x = int(x)
    except:
        x = np.nan
    return x

In [6]:
#Every column except the last column is converted to type int
for column in data.columns[:-1]:
    data[column] = data[column].map(to_number)

In [7]:
data

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,class
0,5,1,1,1,2,1.0,3,1,1,class1
1,5,4,4,5,7,10.0,3,2,1,class1
2,3,1,1,1,2,2.0,3,1,1,class1
3,6,8,8,1,3,4.0,3,7,1,class1
4,4,1,1,3,2,1.0,3,1,1,class1
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2.0,1,1,1,class1
695,2,1,1,1,2,1.0,1,1,1,class1
696,5,10,10,3,7,3.0,8,10,2,class2
697,4,8,6,4,3,4.0,10,6,1,class2


In [ ]:
# Pre-process dataset


In [8]:
#See if '?' is replaced with np.nan
data['Bare Nuclei'].unique()

array([ 1., 10.,  2.,  4.,  3.,  9.,  7., nan,  5.,  8.,  6.])

In [9]:
#Replace class1 and class2 with 0 and 1
data = data.replace(to_replace="class1",value=0)
data = data.replace(to_replace="class2",value=1)

In [10]:
#Using SimpleImputer, replace np.nan with mean
transfer = SimpleImputer()
trans_data = transfer.fit_transform(data)

In [11]:
#Convert to DataFrame format
result_data = pd.DataFrame(trans_data, columns=data.columns)

In [12]:
#Convert all data to int
result_data = result_data.applymap(int)

In [13]:
result_data

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2,1,1,1,0
695,2,1,1,1,2,1,1,1,1,0
696,5,10,10,3,7,3,8,10,2,1
697,4,8,6,4,3,4,10,6,1,1


In [ ]:
#Normalising the data


In [14]:
minmaxTransfer = MinMaxScaler(feature_range=(0,1),copy=1)
minMax_data = minmaxTransfer.fit_transform(result_data)

In [15]:
minMax_data

array([[0.44444444, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.44444444, 0.33333333, 0.33333333, ..., 0.11111111, 0.        ,
        0.        ],
       [0.22222222, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.44444444, 1.        , 1.        , ..., 1.        , 0.11111111,
        1.        ],
       [0.33333333, 0.77777778, 0.55555556, ..., 0.55555556, 0.        ,
        1.        ],
       [0.33333333, 0.77777778, 0.77777778, ..., 0.33333333, 0.        ,
        1.        ]])

In [16]:
#Convert to DataFrame format
minMax_df = pd.DataFrame(minMax_data, columns=data.columns)

In [17]:
#Keep the normalized data to 4 decimal places.
for column in minMax_df.columns[:-1]:
    minMax_df[column] = minMax_df[column].map(lambda x: round(x, 4))

In [18]:
#Cast class to int type
minMax_df['class'] = minMax_df['class'].map(int)

In [19]:
# Print first ten rows of pre-processed dataset to 4 decimal places
minMax_df

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,class
0,0.4444,0.0000,0.0000,0.0000,0.1111,0.0000,0.2222,0.0000,0.0000,0
1,0.4444,0.3333,0.3333,0.4444,0.6667,1.0000,0.2222,0.1111,0.0000,0
2,0.2222,0.0000,0.0000,0.0000,0.1111,0.1111,0.2222,0.0000,0.0000,0
3,0.5556,0.7778,0.7778,0.0000,0.2222,0.3333,0.2222,0.6667,0.0000,0
4,0.3333,0.0000,0.0000,0.2222,0.1111,0.0000,0.2222,0.0000,0.0000,0
...,...,...,...,...,...,...,...,...,...,...
694,0.2222,0.0000,0.0000,0.0000,0.2222,0.1111,0.0000,0.0000,0.0000,0
695,0.1111,0.0000,0.0000,0.0000,0.1111,0.0000,0.0000,0.0000,0.0000,0
696,0.4444,1.0000,1.0000,0.2222,0.6667,0.2222,0.7778,1.0000,0.1111,1
697,0.3333,0.7778,0.5556,0.3333,0.2222,0.3333,1.0000,0.5556,0.0000,1


In [20]:
data = minMax_df.copy()

In [21]:

data.head(10)

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,class
0,0.4444,0.0000,0.0000,0.0000,0.1111,0.0000,0.2222,0.0000,0.0000,0
1,0.4444,0.3333,0.3333,0.4444,0.6667,1.0000,0.2222,0.1111,0.0000,0
2,0.2222,0.0000,0.0000,0.0000,0.1111,0.1111,0.2222,0.0000,0.0000,0
3,0.5556,0.7778,0.7778,0.0000,0.2222,0.3333,0.2222,0.6667,0.0000,0
4,0.3333,0.0000,0.0000,0.2222,0.1111,0.0000,0.2222,0.0000,0.0000,0
5,0.7778,1.0000,1.0000,0.7778,0.6667,1.0000,0.8889,0.6667,0.0000,1
6,0.0000,0.0000,0.0000,0.0000,0.1111,1.0000,0.2222,0.0000,0.0000,0
7,0.1111,0.0000,0.1111,0.0000,0.1111,0.0000,0.2222,0.0000,0.0000,0
8,0.1111,0.0000,0.0000,0.0000,0.1111,0.0000,0.0000,0.0000,0.4444,0
9,0.3333,0.1111,0.0000,0.0000,0.1111,0.0000,0.1111,0.0000,0.0000,0


### Part 1: Cross validation without parameter tuning

In [22]:
## Setting the 10 fold stratified cross-validation
cvKFold=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# The stratified folds from cvKFold should be provided to the classifiers

In [23]:
# K-Nearest Neighbour

#Define an array, receive the accuracy of each calculation, and return the average of the accuracy
def kNNClassifier(X, y, k):
    scores = []
    
    for train_index, test_index in cvKFold.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        mark = knn.score(X_test, y_test)
        scores.append(mark)
    scores = np.array(scores)
    return scores.mean()

In [24]:
# Logistic Regression

#Define an array, receive the accuracy of each calculation, and return the average of the accuracy
def logregClassifier(X, y):
    scores = []
    for train_index, test_index in cvKFold.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        logreg = LogisticRegression()
        logreg.fit(X_train, y_train)
        mark = logreg.score(X_test, y_test)
        scores.append(mark)
    scores = np.array(scores)   
    return scores.mean()

In [25]:
#Naïve Bayes

#Cross-validation with cross_val_score
def nbClassifier(X, y):
    
    nb = GaussianNB()
    scores = cross_val_score(nb, X, y, cv=cvKFold)
    return scores.mean()

In [26]:
# Decision Tree

#Define an array, receive the accuracy of each calculation, and return the average of the accuracy
def dtClassifier(X, y):
    scores = []
    for train_index, test_index in cvKFold.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        tree = DecisionTreeClassifier(criterion='entropy', random_state=0)
        tree.fit(X_train, y_train)
        mark = tree.score(X_train, y_train)
        scores.append(mark)
    scores = np.array(scores) 
    return scores.mean()

In [27]:
# Ensembles: Bagging, Ada Boost and Gradient Boosting


#Define an array, receive the accuracy of each calculation, and return the average of the accuracy
def bagDTClassifier(X, y, n_estimators, max_samples, max_depth):
    scores = []
    for train_index, test_index in cvKFold.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        bag_clf = BaggingClassifier(
            DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth = max_depth), n_estimators=n_estimators, 
            max_samples=max_samples, bootstrap=True, random_state=0)
        bag_clf.fit(X_train, y_train)
        y_pred = bag_clf.predict(X_test)
        mark = accuracy_score(y_test, y_pred)
        scores.append(mark)
    scores = np.array(scores) 
    return scores.mean()
#Define an array, receive the accuracy of each calculation, and return the average of the accuracy
def adaDTClassifier(X, y, n_estimators, learning_rate, max_depth):
    scores = []
    for train_index, test_index in cvKFold.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        ada_clf = AdaBoostClassifier(
            DecisionTreeClassifier(criterion='entropy', max_depth=max_depth), n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
        ada_clf.fit(X_train, y_train)
        y_pred_ada = ada_clf.predict(X_test)
        mark = accuracy_score(y_test, y_pred_ada)
        scores.append(mark)
    
    scores = np.array(scores) 
    return scores.mean()
#Define an array, receive the accuracy of each calculation, and return the average of the accuracy
def gbClassifier(X, y, n_estimators, learning_rate):
    scores = []
    for train_index, test_index in cvKFold.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        gb_clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
        gb_clf.fit(X_train, y_train)
        y_pred_gb = gb_clf.predict(X_test)
        mark = accuracy_score(y_test, y_pred_gb)
        scores.append(mark)
    scores = np.array(scores) 
    return scores.mean()

### Part 1 Results

In [28]:
# Parameters for Part 1:
#KNN
k=3

#Bagging
bag_n_estimators = 50
bag_max_samples = 100
bag_max_depth = 5

#AdaBoost
ada_n_estimators = 50
ada_learning_rate = 0.5
ada_bag_max_depth = 5

#GB
gb_n_estimators = 50
gb_learning_rate = 0.5
#Divide X and y
X=data.iloc[:, 0:-1].values
y=data["class"].values

# Print results for each classifier in part 1 to 4 decimal places here:
print("kNN average cross-validation accuracy:{:.4f} ".format(kNNClassifier(X, y, k)))
print("LR average cross-validation accuracy:{:.4f} ".format(logregClassifier(X, y)))
print("NB average cross-validation accuracy:{:.4f} ".format(nbClassifier(X, y)))
print("DT average cross-validation accuracy:{:.4f} ".format(dtClassifier(X, y)))
print("Bagging average cross-validation accuracy:{:.4f} ".format(bagDTClassifier(X, y, bag_n_estimators, bag_max_samples , bag_max_depth)))
print("AdaBoost average cross-validation accuracy:{:.4f} ".format(adaDTClassifier(X, y, ada_n_estimators, ada_learning_rate, ada_bag_max_depth)))
print("GB average cross-validation accuracy:{:.4f} ".format(gbClassifier(X, y, gb_n_estimators, gb_learning_rate)))

kNN average cross-validation accuracy:0.9628 
LR average cross-validation accuracy:0.9642 
NB average cross-validation accuracy:0.9585 
DT average cross-validation accuracy:1.0000 
Bagging average cross-validation accuracy:0.9542 
AdaBoost average cross-validation accuracy:0.9614 
GB average cross-validation accuracy:0.9599 


### Part 2: Cross validation with parameter tuning

In [29]:


cvKFold=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
# Linear SVM
# You should use SVC from sklearn.svm

#Use GridSearchCV for model selection and tuning, and finally return the object instantiated by GridSearchCV 
#to facilitate retrieval of information such as the best parameters.
C = [0.001, 0.01, 0.1, 1, 10, 100] 
gamma = [0.001, 0.01, 0.1, 1, 10, 100] 
def bestLinClassifier(X,y):
    x_train,x_test,y_train,y_test = train_test_split (X,y,random_state=0)
    parameter_grid = {'C':C,'gamma':gamma}
    grid_search = GridSearchCV(SVC(kernel="linear"),
                           param_grid=parameter_grid,
                           cv=cvKFold)
    
    grid_search.fit(x_train,y_train)
    
    return grid_search  #(appropriate values so that the required printing can be done)

In [30]:

# Random Forest
# You should use RandomForestClassifier from sklearn.ensemble with information gain and max_features set to ‘sqrt’.
n_estimators = [10, 20, 30, 50, 100]
max_leaf_nodes = [4, 10, 16, 20, 30]

#Use GridSearchCV for model selection and tuning, and finally return the object instantiated by GridSearchCV 
#to facilitate retrieval of information such as the best parameters.

def bestRFClassifier(X,y):
    x_train,x_test,y_train,y_test = train_test_split (X,y,random_state=0)
    parameter_grid = {'n_estimators':n_estimators,'max_leaf_nodes':max_leaf_nodes}
    grid_search = GridSearchCV(RandomForestClassifier(criterion='entropy', random_state=0),
                           param_grid=parameter_grid,
                           cv=cvKFold)
    grid_search.fit(x_train,y_train)
    return grid_search #(appropriate values so that the required printing can be done)

### Part 2 Results

In [31]:
# Perform Grid Search with 10-fold Stratified Cross Validation (GridSearchCV in sklearn). 
# The stratified folds from cvKFold should be provided to GridSearchV

# This should include using train_test_split from sklearn.model_selection with stratification and random_state=0
# Print results for each classifier here. All results should be printed to 4 decimal places except for
# "n_estimators" and "max_leaf_nodes" which should be printed as integers.
#Divide X and y
X=data.iloc[:, 0:-1].values
y=data["class"].values
x_train,x_test,y_train,y_test = train_test_split (X,y,random_state=0)

print("SVM best C:{:.4f} ".format(bestLinClassifier(X,y).best_params_.get("C")))
print("SVM best gamma:{:.4f} ".format(bestLinClassifier(X,y).best_params_.get("gamma")))
print("SVM cross-validation accuracy:{:.4f} ".format(bestLinClassifier(X,y).best_score_))
print("SVM test set accuracy:{:.4f} ".format(bestLinClassifier(X,y).score(x_test,y_test)))

print("RF best n_estimators:{} ".format(bestRFClassifier(X,y).best_params_.get("n_estimators")))
print("RF best max_leaf_nodes:{} ".format(bestRFClassifier(X,y).best_params_.get("max_leaf_nodes")))
print("RF cross-validation accuracy:{:.4f} ".format(bestRFClassifier(X,y).best_score_))
print("RF test set accuracy:{:.4f} ".format(bestRFClassifier(X,y).score(x_test,y_test)))

SVM best C:1.0000 
SVM best gamma:0.0010 
SVM cross-validation accuracy:0.9695 
SVM test set accuracy:0.9600 
RF best n_estimators:100 
RF best max_leaf_nodes:16 
RF cross-validation accuracy:0.9695 
RF test set accuracy:0.9657 
